<a href="https://colab.research.google.com/github/z1kurat/Anamorphosis-Cylinder/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers
!pip install transformers==4.30

Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [ ]:
import transformers
import torch
print("Transformers version:", transformers.__version__)
print("PyTorch version:", torch.__version__)

Transformers version: 4.30.0
PyTorch version: 2.4.0+cu121


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset
import csv


In [5]:
filtered_df = pd.read_csv(
    '30_cats.csv',
    encoding="utf-8",
    on_bad_lines='skip',
    sep = '\t',
    engine='python'
    )

In [6]:
filtered_df.shape

(68358, 4)

In [7]:
filtered_df.head()

,Unnamed: 0.1,Unnamed: 0,text,label
0,0,0.0,"**Китай тестирует пушку Гаусса, которая разгон...",5) Технологии
1,1,1.0,🤟 **Недельный объем торгов NFT **[**упал](http...,8) Криптовалюты
2,2,2.0,"Кольцо пара из вулкана Этна, Италия, **попало ...",23) Музыка
3,3,3.0,Путин поручил вернуть в школьную программу про...,16) Познавательное
4,4,4.0,Тётеньке 54 года,1) Другое


In [8]:
original_classes = [
    '1) Другое',
    '2) Блоги',
    '3) Новости и СМИ',
    '4) Юмор и развлечения',
    '5) Технологии',
    '6) Экономика',
    '7) Бизнес и стартапы',
    '8) Криптовалюты',
    '9) Путешествия',
    '10) Маркетинг, PR, реклама',
    '11) Психология',
    '12) Дизайн',
    '13) Политика',
    '14) Искусство',
    '15) Право',
    '16) Образование',
    '17) Книги',
    '18) Лингвистика',
    '19) Карьера',
    '20) Познавательное',
    '21) Курсы и гайды',
    '22) Спорт',
    '23) Мода и красота',
    '24) Медицина',
    '25) Здоровье и фитнес',
    '26) Картинки и фото',
    '27) Софт и приложения',
    '28) Видео и фильмы',
    '29) Музыка',
    '30) Игры',
    '31) Еда и кулинария',
    '32) Цитаты',
    '33) Рукоделие',
    '34) Финансы',
    '35) Технологии',
    '36) Политика',
    '37) Шоубиз',
    '38) Fashion',
    '39) Криптотехнологии',
    '40) Путешествия/релокация',
    '41) Образовательный контент',
    '42) Развлечения',
    '30) Спам-реклама'
]

new_classes = [
    '1) Другое',
    '2) Блоги',
    '3) Новости и СМИ',
    '4) Развлечения и юмор',
    '5) Технологии',
    '6) Экономика',
    '7) Бизнес и стартапы',
    '8) Криптовалюты',
    '9) Путешествия',
    '10) Маркетинг и реклама',
    '11) Психология',
    '12) Дизайн',
    '13) Политика',
    '14) Искусство',
    '15) Право',
    '16) Познавательное',
    '17) Спорт',
    '18) Мода и красота',
    '19) Здоровье и медицина',
    '20) Картинки и фото',
    '21) Софт и приложения',
    '22) Видео и фильмы',
    '23) Музыка',
    '24) Игры',
    '25) Еда и кулинария',
    '26) Цитаты',
    '27) Рукоделие',
    '28) Финансы',
    '29) Шоубиз',
    '30) Спам-реклама'
]

mapping = {
    1: [1],
    2: [2],
    3: [3],
    4: [4, 42],
    5: [5, 35],
    6: [6],
    7: [7, 19],
    8: [8, 39],
    9: [9, 40],
    10: [10],
    11: [11],
    12: [12],
    13: [13, 36],
    14: [14],
    15: [15],
    16: [16, 20, 41, 17, 18, 21],
    17: [22],
    18: [23, 38],
    19: [24, 25],
    20: [26],
    21: [27],
    22: [28],
    23: [29],
    24: [30],
    25: [31],
    26: [32],
    27: [33],
    28: [34],
    29: [37],
    30: [43]
}

In [9]:
inversed_mapping = {}
for k, v in mapping.items():
    for t in v:
        inversed_mapping[t] = k

string_mapping = {}
for k, v in inversed_mapping.items():
    string_mapping[original_classes[k-1]] = new_classes[v-1]

Ячейку ниже не выполнять...

In [10]:
filtered_df['label'] = filtered_df.label.map(lambda x: string_mapping[x])

KeyError: '23) Музыка'

In [11]:
filtered_df.label.value_counts()

label
13) Политика               5698
8) Криптовалюты            3687
5) Технологии              3397
3) Новости и СМИ           3100
9) Путешествия             2964
16) Познавательное         2839
4) Развлечения и юмор      2678
6) Экономика               2671
22) Видео и фильмы         2430
19) Здоровье и медицина    2279
21) Софт и приложения      2231
2) Блоги                   2083
26) Цитаты                 2062
30) Спам-реклама           2024
23) Музыка                 2010
10) Маркетинг и реклама    1952
12) Дизайн                 1945
27) Рукоделие              1933
20) Картинки и фото        1864
18) Мода и красота         1853
28) Финансы                1807
25) Еда и кулинария        1761
11) Психология             1737
14) Искусство              1709
29) Шоубиз                 1708
7) Бизнес и стартапы       1706
17) Спорт                  1705
24) Игры                   1685
15) Право                  1660
1) Другое                  1180
Name: count, dtype: int64

In [12]:
filtered_df.head()

,Unnamed: 0.1,Unnamed: 0,text,label
0,0,0.0,"**Китай тестирует пушку Гаусса, которая разгон...",5) Технологии
1,1,1.0,🤟 **Недельный объем торгов NFT **[**упал](http...,8) Криптовалюты
2,2,2.0,"Кольцо пара из вулкана Этна, Италия, **попало ...",23) Музыка
3,3,3.0,Путин поручил вернуть в школьную программу про...,16) Познавательное
4,4,4.0,Тётеньке 54 года,1) Другое


In [13]:
for x in filtered_df[filtered_df.label == '15) Право'].values:
    print(x[0])
    print('---------------------------------------------__')

10
---------------------------------------------__
126
---------------------------------------------__
226
---------------------------------------------__
478
---------------------------------------------__
1979
---------------------------------------------__
3083
---------------------------------------------__
3138
---------------------------------------------__
3537
---------------------------------------------__
4782
---------------------------------------------__
4969
---------------------------------------------__
5010
---------------------------------------------__
5049
---------------------------------------------__
5067
---------------------------------------------__
5977
---------------------------------------------__
6809
---------------------------------------------__
6880
---------------------------------------------__
7264
---------------------------------------------__
7935
---------------------------------------------__
8048
---------------------------------------------_

In [14]:
print(filtered_df.label.nunique())
print(filtered_df.label.unique())

30
['5) Технологии' '8) Криптовалюты' '23) Музыка' '16) Познавательное'
 '1) Другое' '3) Новости и СМИ' '22) Видео и фильмы' '6) Экономика'
 '15) Право' '17) Спорт' '19) Здоровье и медицина' '4) Развлечения и юмор'
 '9) Путешествия' '26) Цитаты' '13) Политика' '20) Картинки и фото'
 '11) Психология' '12) Дизайн' '21) Софт и приложения' '28) Финансы'
 '18) Мода и красота' '10) Маркетинг и реклама' '2) Блоги'
 '7) Бизнес и стартапы' '24) Игры' '29) Шоубиз' '14) Искусство'
 '25) Еда и кулинария' '27) Рукоделие' '30) Спам-реклама']


In [15]:
class_to_idx = {name:i for i, name in enumerate(new_classes)}
idx_to_class = {v:k for k, v in class_to_idx.items()}

In [16]:
from transformers import AutoTokenizer

In [17]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
dataset = Dataset.from_pandas(filtered_df)

In [19]:
def preprocess_function(batch):
    batch['label'] = [class_to_idx[class_name] for class_name in batch['label']]
    return tokenizer(batch["text"], truncation=True)

In [20]:
dataset = dataset.map(preprocess_function, batched=True, batch_size=10)

Map:   0%|          | 0/68358 [00:00<?, ? examples/s]

In [21]:
random_idxes = np.random.randint(0, len(dataset), size=10)
for idx in random_idxes:
    idx = int(idx)
    print('TEXT: ')
    print(tokenizer.decode(dataset[idx]['input_ids']))
    print('\n')
    print('LABEL:')
    print(idx_to_class[dataset[idx]['label']])
    print('====================================================')

TEXT: 
[CLS] Британия официально объявила группу " Вагнер " террористической организацией и внесла ее в список запрещенных объединений [SEP]


LABEL:
13) Политика
TEXT: 
[CLS] К международному женскому дню в кинотеатре " Мир " состоялся показ фильма для участников клуба " Активное долголетие ". С праздником прекрасных дам поздравил Председатель Совета депутатов городского округа Шаховская Александр Сергеевич Голованов.. [SEP]


LABEL:
20) Картинки и фото
TEXT: 
[CLS] Кыстыбый относится к татарской, башкирской и чувашской кухням. Это вкусный плоский хлеб с начинкой из картофельного пюре, каши, тыквенного пюре и толченых семян мака.. [SEP]


LABEL:
25) Еда и кулинария
TEXT: 
[CLS] USD / JPY. Итоги январского заседания Банка Японии разочаровали медведей пары, но иену ещё рано списывать со счетов Подробнее на : [SEP]


LABEL:
6) Экономика
TEXT: 
[CLS] ❗️ * * Путин и Лукашенко начали переговоры в Сочи. * * [SEP]


LABEL:
13) Политика
TEXT: 
[CLS] На Украине в школах из учебной программы иск

In [22]:
splitted = dataset.train_test_split(test_size=0.05)
train_dataset = splitted['train']
test_dataset = splitted['test']

In [23]:
print(f'Train dataset len: {len(train_dataset)}')
print(f'Test dataset len: {len(test_dataset)}')

Train dataset len: 64940
Test dataset len: 3418


In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00


In [26]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(
    "sentence-transformers/LaBSE", num_labels=len(idx_to_class), id2label=idx_to_class, label2id=class_to_idx
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir="classifier_logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1.0,
    weight_decay=0.001,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=3,
    logging_steps=100,
    push_to_hub=False,
    fp16=True,
    report_to='tensorboard'
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,1.286300,1.414406,0.629608


In [ ]:
import pandas as pd

In [ ]:
df_gpt4 = pd.read_csv('filtered_gpt4_30.csv')

In [ ]:
df_gpt4.sample(100).to_csv('sample100.csv')

In [ ]:
df_gpt4 = df_gpt4[df_gpt4.label.isin(new_classes)][['text', 'label']]

In [ ]:
dataset_gpt4 = Dataset.from_pandas(df_gpt4)

In [ ]:
dataset_gpt4 = dataset_gpt4.map(preprocess_function, batched=True, batch_size=1000)

In [ ]:
trainer.evaluate(dataset_gpt4)

In [ ]:
splitted_gpt4 = dataset_gpt4.train_test_split(test_size=0.05)
train_dataset_gpt4 = splitted_gpt4['train']
test_dataset_gpt4 = splitted_gpt4['test']

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_gpt4,
    eval_dataset=test_dataset_gpt4,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset_gpt4)

In [ ]:
trainer.save_model('/drive/final_classifier')

In [ ]:
model.cpu()
model.eval()

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model='/drive/final_classifier')

In [ ]:
len(df_gpt4.label.value_counts())

In [ ]:
!pip install optimum[exporters]

Это просто для эмбеддингов в их чистом виде (можно не выполнять)

In [ ]:
!optimum-cli export onnx --model cointegrated/rubert-tiny rubert-tiny_onnx/

А вот это уже для нужного классификатора - в результате создастся папка с 6 файлами

In [ ]:
!optimum-cli export onnx --model final_classifier --task text-classification news_classifier_onnx/

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
